In [1]:
from pandas_datareader import data, wb
import pandas as pd
import numpy as np
import datetime

In [2]:
start = datetime.datetime(2013,1,1)
end = datetime.datetime(2016,3,20)

In [3]:
def label_condition(row):
    if row['Close'] > row['CloseMA'] * 1.02:
        return 'Bull'
    if row['Close'] < row['CloseMA'] * 0.98:
        return 'Bear'
    return 'Sideway'

In [4]:
tsx = data.DataReader("INDEXTSI:OSPTX", 'google', start, end)   #TSX index
atr = tsx.copy()

In [5]:
tsx

,Open,High,Low,Close,Volume
Date,,,,,
2013-01-02,12571.76,12588.75,12439.25,12540.77,149633612
2013-01-03,12539.14,12540.58,12430.41,12470.44,160667711
2013-01-04,12435.12,12540.81,12442.84,12540.81,133008148
2013-01-07,12503.24,12540.81,12448.21,12499.55,152632689
2013-01-08,12490.37,12504.81,12456.00,12504.81,168870483
2013-01-09,12508.13,12528.86,12484.80,12522.24,166360109
2013-01-10,12571.95,12618.43,12523.56,12599.74,207719329
2013-01-11,12600.90,12603.15,12545.03,12602.18,155720308
2013-01-14,12627.41,12636.68,12582.15,12603.09,216433253


In [7]:
tsx_ma20 = pd.rolling_mean(tsx, 200, 200)

In [8]:
tsx_ma20 = tsx_ma20.rename(columns={'Close': 'CloseMA'})

In [9]:
# select sub columns
tsx = tsx[['Close']]
tsx_ma20 = tsx_ma20[['CloseMA']]

In [10]:
tsx = tsx.join(tsx_ma20)
# take only the one with CLoseMA value
tsx = tsx[np.isfinite(tsx['CloseMA'])]

In [11]:
tsx['MarketCondition'] = tsx.apply(lambda row: label_condition(row), axis=1)
#tsx.ix['2015-01-02']
 

In [12]:
atr = atr[['High', 'Low', 'Close']]


In [13]:
atr_prev = atr.shift().rename(columns={'Close': 'ClosePrev'})
atr_prev = atr_prev[['ClosePrev']]

In [14]:
atr = atr.join(atr_prev)
atr = atr[np.isfinite(atr['ClosePrev'])]

In [15]:
def get_avg_true_range(df, period=14):
    df['tr1'] = df['High'] - df['Low']
    df['tr2'] = abs(df['High'] - df['ClosePrev'])
    df['tr3'] = abs(df['Low'] - df['ClosePrev'])
    df['TrueRange'] = df[['tr1','tr2','tr3']].max(axis=1)
    temp_df = pd.rolling_mean(df, period, period)
    temp_df = temp_df[['TrueRange']].rename(columns={'TrueRange':'AvgTrueRange'})
    return temp_df[['AvgTrueRange']]

In [16]:
atr = get_avg_true_range(atr)
atr_ma = pd.rolling_mean(atr, 100, 100).rename(columns={'AvgTrueRange': 'AvgTrueRangeMA'})
atr_ma = atr_ma[np.isfinite(atr_ma['AvgTrueRangeMA'])]
atr_ma_std = pd.rolling_std(atr, 100, 100).rename(columns={'AvgTrueRange': 'AvgTrueRangeMAStd'})
atr_ma_std = atr_ma_std[np.isfinite(atr_ma_std['AvgTrueRangeMAStd'])]
atr = atr[np.isfinite(atr['AvgTrueRange'])]


In [17]:
atr = atr.join(atr_ma, how='inner').join(atr_ma_std, how='inner')

In [18]:
def label_volatility(row):
    if row['AvgTrueRange'] - row['AvgTrueRangeMA'] > row['AvgTrueRangeMAStd']:
        return 'Volatile'
    if row['AvgTrueRangeMA'] - row['AvgTrueRange'] > row['AvgTrueRangeMAStd']:
        return 'Quiet'
    return 'Normal'

In [19]:
atr['MarketVolatility'] = atr.apply(lambda row: label_volatility(row), axis=1)


In [21]:
tsx.join(atr, how='inner')